# 코드 실행 전 CSV 변환, 파일 내 이상한 문구 수정 후 돌릴 것

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.graphics.mosaicplot import mosaic
from scipy import stats as spst
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# 1. 분석할 데이터 경로 및 파일 설정

In [3]:
path = "/content/drive/MyDrive/Colab Notebooks/test.CSV"
data = pd.read_csv(path)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (255) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
# 불러온 파일의 결측지를 NaN이라는 문자로 채우기

data.fillna("NaN",inplace=True)

# 2. 중복 Row 제거 ( 06.21 기준 아직 미완성)

In [ ]:
# 컬럼별 결측지 수 
data[["Q47_1","Q48_2","Q51_2","Q369_2"]].isnull().sum()

Q47_1     964
Q48_2     636
Q51_2     276
Q369_2      2
dtype: int64

In [ ]:
a = data[["Q47_1"]].copy()
b = data[["Q48_2"]].copy()
c = data[["Q51_2"]].copy()
d = data[["Q369_2"]].copy()


# data.drop_duplicates("Q47_1",inplace=True)

In [ ]:
a.drop_duplicates("Q47_1",inplace=True)
b.drop_duplicates("Q48_2",inplace=True)
c.drop_duplicates("Q51_2",inplace=True)
d.drop_duplicates("Q369_2",inplace=True)

In [ ]:
len(a), len(b), len(c), len(d)

(636, 962, 1319, 1569)

In [ ]:
# a, b, c, d에서 컬럼별로 중복 Row 제거한 데이터를 하나로 합치기

result1 = pd.concat([a,b,c,d],axis=1, ignore_index=False)
result1.isnull().sum()

Q47_1     962
Q48_2     636
Q51_2     279
Q369_2     29
dtype: int64

In [ ]:
len(data), len(result1) # 전체 데이터에서 2개만 줄었다는 것은 a,b,c,d가 공통적으로 지워진 Row 부분이 2개 --> 데이터 손실이 매우 미미해서 사용해도 될 듯

(1599, 1597)

# 3. 필요한 컬럼만 뽑아 복사본 만들기

In [22]:
colums = ["index","respondentID","Q47_1","Q48_2","Q51_2","Q369_2"]

copy = data[colums].copy()

In [ ]:
copy

,index,respondentID,Q47_1,Q48_2,Q51_2,Q369_2
0,0,6280,NaN,Experirnced some occasional glitches on the we...,Fix technical glitches. Put up the honest reviews,"Good camera , good battery life. Good processi..."
1,1,64,NaN,Greta overall battery life and camera quality ...,Has good amount of info and details of the pho...,Great battery life and resolution on camera. D...
2,2,127,NaN,Not many products available to purchase and th...,Not many models of Samsung smartphones availab...,Very good experience with my iPhone 11. Fast p...
3,3,148,NaN,Because it's more affordable and it's the perf...,NaN,Very pleased with you the use of my symptom
4,4,232,Very fast in delivery of good and services all...,NaN,It's extremely Good and important in all aspects,Very interesting and important information
...,...,...,...,...,...,...
1594,1594,8288,NaN,"I PLEASED my experience, it was very pleasant ...",I graduated regardless of the score I obtained...,"EXCELLENT, VERY Good equipment, its characteri..."
1595,1595,8400,NaN,Twitter is not the basis for my recommendation...,I really do not have many expectations when I ...,I have had a blast using this phone. It is ver...
1596,1596,8660,NaN,The estimated delivery window after purchasing...,"From what I recall, it listed basically every ...",I miss the home button. I don't want to use fa...
1597,1597,8675,They are very customer oriented. You make an a...,NaN,I am not a aware of any promotions from apple....,I love my iPhone 11. From day 1 (after the sho...


# 4. 유사한 답변을 한 colum들을 뽑아 row 삭제

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
# 삭제한 ROW의 respondentID, Q47_1, Q48_2, Q369_2 값 저장
delete_data = {
    "respondentID" : [],
    "Q47_1" : [],
    "Q48_2" : [],
    "Q51_2" : [],
    "Q369_2" : []
}

A = copy["Q47_1"].values
A = A.tolist()
B = copy["Q48_2"].values
B = B.tolist()
C = copy["Q51_2"].values
C = C.tolist()
D = copy["Q369_2"].values
D = D.tolist()

for i in range(len(copy)):
  mydoclist = [A[i],B[i],C[i],D[i]]

  # 유사도 값 추출과 array 생성
  tfidf_vectorizer = TfidfVectorizer(min_df=1)
  tfidf_matrix = tfidf_vectorizer.fit_transform(mydoclist)
  document_distances = (tfidf_matrix * tfidf_matrix.T)

  # avg = document_distances.toarray()[::1][3][:3].sum() / 3  3개의 유사도의 총합의 평균 --> 한쪽이 유사도가 0.9가 나와도 다른게 0.2, 0.1 나올때 평균이 너무 낮아져서 유사한 답변이라고 취급 안하는 문제 발생
  

  # 4번째 질문을 기준으로 각각의 유사도가 하나라도 기준 유사도(0.6)을 넘으면 같다고 보고 삭제하는 조건문
  if  document_distances.toarray()[0][3] >=0.6 or document_distances.toarray()[1][3]>=0.6 or document_distances.toarray()[2][3] >= 0.6 :
    # 삭제할 index 찾기
    idx = copy[copy["index"] == i ].index

    # 해당 인덱스에 해당하는 colum들 값 을 delete_data에 집어 넣기
    delete_data["respondentID"].extend(copy.loc[:,"respondentID" ][idx].tolist())
    delete_data["Q47_1"].extend(copy.loc[:,"Q47_1" ][idx].tolist())
    delete_data["Q48_2"].extend(copy.loc[:,"Q48_2" ][idx].tolist())
    delete_data["Q51_2"].extend(copy.loc[:,"Q51_2" ][idx].tolist())
    delete_data["Q369_2"].extend(copy.loc[:,"Q369_2" ][idx].tolist())

    # 해당 index의 로우 삭제하고 copy 업데이트
    copy = copy.drop(idx)


# 5. 결과 확인

In [ ]:
# 유사도 뽑기전 데이터는 1599개 였지만 유사도 후 1541로 줄었음
copy

,index,respondentID,Q47_1,Q48_2,Q51_2,Q369_2
0,0,6280,NaN,Experirnced some occasional glitches on the we...,Fix technical glitches. Put up the honest reviews,"Good camera , good battery life. Good processi..."
1,1,64,NaN,Greta overall battery life and camera quality ...,Has good amount of info and details of the pho...,Great battery life and resolution on camera. D...
2,2,127,NaN,Not many products available to purchase and th...,Not many models of Samsung smartphones availab...,Very good experience with my iPhone 11. Fast p...
3,3,148,NaN,Because it's more affordable and it's the perf...,NaN,Very pleased with you the use of my symptom
4,4,232,Very fast in delivery of good and services all...,NaN,It's extremely Good and important in all aspects,Very interesting and important information
...,...,...,...,...,...,...
1593,1593,8149,in my experience search in amazon was excellen...,NaN,is excellent website,i feel satisfied with this brand
1594,1594,8288,NaN,"I PLEASED my experience, it was very pleasant ...",I graduated regardless of the score I obtained...,"EXCELLENT, VERY Good equipment, its characteri..."
1595,1595,8400,NaN,Twitter is not the basis for my recommendation...,I really do not have many expectations when I ...,I have had a blast using this phone. It is ver...
1596,1596,8660,NaN,The estimated delivery window after purchasing...,"From what I recall, it listed basically every ...",I miss the home button. I don't want to use fa...


In [ ]:
len(delete_data["respondentID"]), len(delete_data["Q47_1"]), len(delete_data["Q48_2"]), len(delete_data["Q51_2"]), len(delete_data["Q369_2"])

(58, 58, 58, 58, 58)

# 6. 가공한 데이터 저장

In [ ]:
import pandas as pd

# 중복 제거한 데잍터만 남겨서 따로 저장
data_df = pd.DataFrame(delete_data)
data_df.to_csv("delet_data.csv",columns=["respondentID","Q47_1","Q48_2","Q51_2","Q369_2"],index=True)


# 중복 제거 후 남은 데이터 저장
copy.to_csv("test_finish.csv",columns=["index","respondentID","Q47_1","Q48_2","Q51_2","Q369_2"],index=False)

# 다음 코드부터는 문법 Test 

In [ ]:
import pandas as pd

a = {'A' : [111, 111, 111, 111, 222, 222], 'B' : ['aaa', 'aaa', 'bbb', 'bbb', 'aaa', 'bbb'], 
'C' : ['zzz', 'zzz', 'zzz', 'yyy', 'zzz', 'yyy']}
df = pd.DataFrame(a)
df

,A,B,C
0,111,aaa,zzz
1,111,aaa,zzz
2,111,bbb,zzz
3,111,bbb,yyy
4,222,aaa,zzz
5,222,bbb,yyy


In [ ]:
df.drop_duplicates()

,A,B,C
0,111,aaa,zzz
2,111,bbb,zzz
3,111,bbb,yyy
4,222,aaa,zzz
5,222,bbb,yyy


In [ ]:
delete_data

In [ ]:
data_df = pd.DataFrame(delete_data)

In [ ]:
data_df

,respondentID,Q47_1,Q48_2,Q51_2,Q369_2
0,371,NaN,I like the brand as a whole. They offer great...,I like what the brand has to offer. They offer...,I like what the brand has offer. They offer gr...
1,3926,very,NaN,very good goood,very good
2,2639,It is very very friendly.There service are ver...,NaN,NaN,It is very friendly for use.it is very high qu...
3,3691,NaN,Below you can see all questionnaires which are...,Below you can see all questionnaires which are...,Below you can see all questionnaires which are...
4,3848,NaN,i like it.it is so good.i love it.,i love it.,i love it.
5,5193,very informative and this is so helpful and I...,NaN,apple website is very good and I liked it very...,it is very good
6,3902,it is very smart and famous for me. it is very...,NaN,NaN,it is very easy and comfortable for me. it is ...
7,1016,NaN,T-Mobile's a great deal of time in the past ye...,Good quality,Good
8,1259,Very great and beautiful experience at the sto...,NaN,Nothing anyway... I used to have the best expe...,Very great and beautiful experience at the usi...
9,3984,NaN,good good good good good good good good good g...,good good good,good good good


In [44]:
document_distances.toarray()

array([[1.        , 0.        , 0.78309762, 0.95618289],
       [0.        , 1.        , 0.        , 0.        ],
       [0.78309762, 0.        , 1.        , 0.81898309],
       [0.95618289, 0.        , 0.81898309, 1.        ]])

In [40]:
document_distances.toarray()[1]

array([0., 1., 0., 0.])

In [41]:
document_distances.toarray()[1][3]

0.0